In [1]:
from clipp2.core1 import *
from clipp2.preprocess import *
df1 = process_files('AVPC/ACC9/9-post.snv.txt', 'AVPC/ACC9/9-post.cna.txt','AVPC/ACC9/9-post.purity.txt')
df2 = process_files('AVPC/ACC9/9-pre.snv.txt', 'AVPC/ACC9/9-pre.cna.txt','AVPC/ACC9/9-pre.purity.txt')
[df1, df2] = insert_distinct_rows_multi([df1, df2])
export_snv_cna_and_purity(
        df1,
        dir="AVPC/ACC9pre/",
        snv_path="9post.snv.txt",
        cna_path="9post.cna.txt",
        purity_path="9post.purity.txt"
    )
export_snv_cna_and_purity(
        df2,
        dir="AVPC/ACC9pre/",
        snv_path="9pre.snv.txt",
        cna_path="9pre.cna.txt",
        purity_path="9pre.purity.txt"
    )
snv_file      = "AVPC/ACC9pre/9post.snv.txt"
cn_file       = "AVPC/ACC9pre/9post.cna.txt"
purity_file   = "AVPC/ACC9pre/9post.purity.txt"
sample_id     = "--sample_id"   # not heavily used in logic
output_prefix = "E:/Dropbox/GitHub/Multi_Region_CliPP/processed_data/acc9post"

# Call function with drop_data=True (to replicate R code's dropping logic)
process_data(
    snv_file, cn_file, purity_file, sample_id, output_prefix, 
    drop_data=False
)
snv_file      = "AVPC/ACC9pre/9pre.snv.txt"
cn_file       = "AVPC/ACC9pre/9pre.cna.txt"
purity_file   = "AVPC/ACC9pre/9pre.purity.txt"
sample_id     = "--sample_id"   # not heavily used in logic
output_prefix = "E:/Dropbox/GitHub/Multi_Region_CliPP/processed_data/acc9pre"

Loaded SNV data from AVPC/ACC9/9-post.snv.txt with shape (3862, 4)
Loaded CNA data from AVPC/ACC9/9-post.cna.txt with shape (181, 6)
Purity: 0.21
Constructed df with shape (3862, 11)
Loaded SNV data from AVPC/ACC9/9-pre.snv.txt with shape (5576, 4)
Loaded CNA data from AVPC/ACC9/9-pre.cna.txt with shape (277, 6)
Purity: 0.31
Constructed df with shape (5576, 11)
Process done. Created outputs in E:/Dropbox/GitHub/Multi_Region_CliPP/processed_data/acc9post.


In [2]:
# Call function with drop_data=True (to replicate R code's dropping logic)
process_data(
    snv_file, cn_file, purity_file, sample_id, output_prefix, 
    drop_data=False
)

Process done. Created outputs in E:/Dropbox/GitHub/Multi_Region_CliPP/processed_data/acc9pre.


e:\Dropbox\GitHub\Multi_Region_CliPP\clipp2\preprocess.py:763: RuntimeWarning: divide by zero encountered in divide
  phi_arr = 2.0 / ( (minor_count / (minor_read/total_read)) - total_count + 2.0 )


In [3]:
root_dir = "E:/Dropbox/GitHub/Multi_Region_CliPP/processed_data"
(r, n, minor, total, purity, coef_list, wcut, drop) = group_all_regions_for_ADMM(root_dir)

Loaded region 'acc9post': r.shape=(5793,), coef.shape=(5793, 6), purity=0.21
Loaded region 'acc9pre': r.shape=(5793,), coef.shape=(5793, 6), purity=0.31

=== Summary of grouped data before dropping rows ===
Found M=2 regions. r shape= (5793, 2), n= (5793, 2)
minor= (5793, 2), total= (5793, 2)
coef_list length= 2 (each is (No_mutation,6))
wcut= [-0.18  1.8 ]

Dropped 0 rows that were all-zero in r/n/minor/total/coef.

=== Summary of grouped data after dropping rows ===
r shape= (5793, 2), n= (5793, 2)
minor= (5793, 2), total= (5793, 2)
coef_list length= 2, each => shape (5793, 6)


In [4]:
wcut = [-1.8, 1.8]
alpha=0.8
gamma=3.7
rho=1.02
precision= 0.01
Run_limit=1e4
control_large=5
Lambda=0.1
post_th=0.05
least_diff=0.01

In [5]:
def ensure_2D_and_no_zeros(arr):
    """
    - If arr is 1D, reshape to (No_mutation, 1).
    - If arr is 2D, keep shape.
    - Then replace any zeros with 1.
    """
    if arr.ndim == 1:
        arr = arr.reshape(-1, 1)
    elif arr.ndim != 2:
        raise ValueError(f"Expected 1D or 2D array, got shape {arr.shape}")
    # Replace zeros with 1
    # This is vectorized, no Python loops:
    arr = np.where(arr == 0, 1, arr)
    return arr

r     = ensure_2D_and_no_zeros(r)
n     = ensure_2D_and_no_zeros(n)
minor = ensure_2D_and_no_zeros(minor)
total = ensure_2D_and_no_zeros(total)

No_mutation, M = r.shape

# -------- 2) Broadcast purity => shape (No_mutation, M).  Fix ploidy=2.0. --------
if np.isscalar(purity):
    purity_arr = np.full((No_mutation, M), float(purity))
else:
    purity = np.asarray(purity, dtype=float)
    if purity.ndim == 1 and purity.shape[0] == M:
        purity_arr = np.broadcast_to(purity.reshape(1,M), (No_mutation, M))
    else:
        raise ValueError(f"purity must be scalar or shape (M,). Got shape {purity.shape}.")
ploidy_arr = np.full((No_mutation, M), 2.0)

# -------- 3) Compute w_new, using single-sample style bounding logic --------
# fraction = (r + eps)/(n + 2eps), then scale by copy/purity factor, clip in [expit(-cl), expit(cl)]
fraction = (r ) / (n )
phi_hat  = fraction * ( (ploidy_arr - purity_arr*ploidy_arr) + (purity_arr*total) ) / minor

scale_parameter = max(1.0, np.max(phi_hat))
phi_new = phi_hat / scale_parameter

low_b = expit(-control_large)
up_b  = expit(control_large)
phi_new = np.clip(phi_new, low_b, up_b)  # shape (No_mutation, M)

w_init = logit(phi_new)
w_init = np.clip(w_init, -control_large, control_large)
w_new  = w_init.copy()

# -------- 4) Build the DELTA operator with NO Python for-loops --------
# We want pairs (i<j).  We'll use triu_indices:
i_idx, j_idx = np.triu_indices(No_mutation, k=1)     # shape=(No_pairs,)
No_pairs = i_idx.size                                # # of i<j pairs

# For each pair k, we have M rows in the final big matrix.
# row indices => (k*M + m) for m in [0..M-1].
# We'll build them in bulk.
row_block = np.arange(No_pairs)[:,None]*M + np.arange(M)   # shape=(No_pairs, M)
# We want 2 columns per row block => one for +1 at col i, one for -1 at col j,
# so the total rows = 2 * (No_pairs * M).  But the row index is the same for i and j.
# => We'll horizontally stack row_block with itself, then flatten.
row_block_2 = np.hstack((row_block, row_block))            # shape=(No_pairs, 2*M)
row_idx = row_block_2.ravel()                              # shape=(2*M*No_pairs,)

# For column indices => i_idx[k], j_idx[k] => i_idx[k]*M + m
col_block_i = i_idx[:,None]*M + np.arange(M)   # shape=(No_pairs, M)
col_block_j = j_idx[:,None]*M + np.arange(M)   # shape=(No_pairs, M)
col_block_2 = np.hstack((col_block_i, col_block_j))  # shape=(No_pairs, 2*M)
col_idx = col_block_2.ravel()

# For the data => +1 for i, -1 for j
plus_ones  = np.ones_like(col_block_i)
minus_ones = -np.ones_like(col_block_j)
data_block = np.hstack((plus_ones, minus_ones))    # shape=(No_pairs, 2*M)
vals = data_block.ravel()                          # shape=(2*M*No_pairs,)

total_rows = No_pairs * M
total_cols = No_mutation * M
DELTA = sp.coo_matrix((vals, (row_idx, col_idx)),
                        shape=(total_rows, total_cols)).tocsr()

# -------- 5) Initialize eta, tau, residual, iteration k with NO loops --------
# We can get the initial pairwise differences: w_new[i_idx,:] - w_new[j_idx,:]
# shape => (No_pairs, M)
eta_new = w_new[i_idx, :] - w_new[j_idx, :]

# tau_new => just ones
tau_new = np.ones_like(eta_new)

# Large initial residual
residual = 1e6
k = 0

c_all = np.stack(coef_list, axis=1)
i_idx, j_idx = np.triu_indices(No_mutation, k=1)  
No_pairs = i_idx.size

In [ ]:
while k < Run_limit and residual > precision:
    
    k += 1    

    w_old  = w_new.copy()
    eta_old = eta_new.copy()
    tau_old = tau_new.copy()

    # =========================
    # (A) IRLS expansions in bulk
    # =========================

    # 1) theta = (e^w * minor) / (2 + e^w * total)
    expW   = np.exp(w_old)
    denom_ = 2.0 + (expW*total)
    # avoid zero denominators
    denom_ = np.where(denom_==0, 1e-12, denom_)
    theta  = (expW*minor)/denom_    # shape => (No_mutation, M)

    # 2) Build partA, partB fully vectorized
    #    shape => (No_mutation, M)
    #    using c_all => shape(No_mutation, M, 6)
    low_cut, up_cut = wcut
    maskLow =  (w_old <= low_cut)
    maskUp  =  (w_old >= up_cut)
    maskMid = ~(maskLow | maskUp)

    # partA_full = (branch on c_all[...,1], c_all[...,3], c_all[...,5]) 
    #              minus (r/n)
    partA_full = (
        (maskLow * c_all[...,1])
        + (maskUp  * c_all[...,5])
        + (maskMid * c_all[...,3])
    ) - (r / n)

    # partB_full similarly from c_all[...,0], c_all[...,2], c_all[...,4]
    partB_full = (
        (maskLow * c_all[...,0])
        + (maskUp  * c_all[...,4])
        + (maskMid * c_all[...,2])
    )

    # 3) A_array, B_array => multiply by sqrt(n) / sqrt(theta*(1-theta))
    sqrt_n = np.sqrt(n)
    # avoid zero => add small epsilon
    denom2 = np.sqrt(theta*(1 - theta) + 1e-12)

    A_array = (sqrt_n * partA_full) / denom2
    B_array = (sqrt_n * partB_full) / denom2

    # 4) Flatten them for the linear system
    A_flat = A_array.ravel()    # shape => (No_mutation*M,)
    B_flat = B_array.ravel()

    # =========================
    # (B) Form the linear system => (B^T B + alpha Delta^T Delta) w = ...
    # =========================
    # big_eta_tau => alpha*eta_old + tau_old
    big_eta_tau = alpha*eta_old + tau_old
    big_eta_tau_flat = big_eta_tau.ravel()

    linear_1 = DELTA.transpose().dot(big_eta_tau_flat)
    # linear_2 = B_flat * A_flat => elementwise, shape => (No_mutation*M,)
    linear_2 = B_flat * A_flat
    # final 'linear'
    linear   = linear_1 - linear_2

    # B^T B => diag of B_flat^2
    B_sq = B_flat**2
    Bmat = sp.diags(B_sq, 0, shape=(No_mutation*M, No_mutation*M))
    H    = Bmat + alpha*(DELTA.transpose().dot(DELTA))

    # Solve
    w_new_flat = spsolve(H, linear)
    w_new = w_new_flat.reshape(No_mutation, M)
    # clip
    np.clip(w_new, -control_large, control_large, out=w_new)

    # =========================
    # (C) SCAD threshold => eta_new, tau_new (already vectorized)
    # =========================
    eta_new, tau_new = scad_threshold_update(
        w_new, tau_old, DELTA, alpha, Lambda, gamma
    )

    # scale alpha
    alpha *= rho

    # =========================
    # (D) residual check in bulk (no for-loops)
    # =========================
    #  Residual = max_{pair} max_{m} | (w_new[i,:]-w_new[j,:]) - eta_new[k,:] |
    #
    # i_idx, j_idx => shape (No_pairs,)
    # => compute w_new[i_idx,:] - w_new[j_idx,:], shape => (No_pairs, M)
    # subtract eta_new => shape => (No_pairs, M)
    diff_2D = (w_new[i_idx,:] - w_new[j_idx,:]) - eta_new
    # compute max absolute difference
    residual = np.max(np.abs(diff_2D))

    print(f"Iteration={k}, alpha={alpha:.4g}, residual={residual:.6g}")

Iteration=1, alpha=0.816, residual=1.375
Iteration=2, alpha=0.8323, residual=0.188225
Iteration=3, alpha=0.849, residual=0.120146
Iteration=4, alpha=0.8659, residual=0.0879087
Iteration=5, alpha=0.8833, residual=0.0667848
Iteration=6, alpha=0.9009, residual=0.0483592
Iteration=7, alpha=0.9189, residual=0.0343762
Iteration=8, alpha=0.9373, residual=0.0269698
Iteration=9, alpha=0.9561, residual=0.0233618
Iteration=10, alpha=0.9752, residual=0.0212269
Iteration=11, alpha=0.9947, residual=0.0172917
Iteration=12, alpha=1.015, residual=0.0165185
Iteration=13, alpha=1.035, residual=0.0150461
Iteration=14, alpha=1.056, residual=0.0143854
Iteration=15, alpha=1.077, residual=0.0149511
Iteration=16, alpha=1.098, residual=0.0114916
Iteration=17, alpha=1.12, residual=0.0108306
Iteration=18, alpha=1.143, residual=0.0106503
Iteration=19, alpha=1.165, residual=0.0111917
Iteration=20, alpha=1.189, residual=0.0108229
Iteration=21, alpha=1.213, residual=0.0110554
Iteration=22, alpha=1.237, residual=0.010

In [ ]:
print("\nADMM finished.\n")

diff = diff_mat(w_new)
ids = np.triu_indices(diff.shape[1], 1)
eta_new[np.where(np.abs(eta_new) <= post_th)] = 0
diff[ids] = np.linalg.norm(eta_new, axis=1)
class_label = -np.ones(No_mutation)
class_label[0] = 0
group_size = [1]
labl = 1

for i in range(1, No_mutation):
    for j in range(i):
        if diff[j, i] == 0:
            class_label[i] = class_label[j]
            group_size[int(class_label[j])] += 1
            break
    if class_label[i] == -1:
        class_label[i] = labl
        labl += 1
        group_size.append(1)

# quality control
least_mut = np.ceil(0.05 * No_mutation)
tmp_size = np.min(np.array(group_size)[np.array(group_size) > 0])
tmp_grp = np.where(group_size == tmp_size)
refine = False
if tmp_size < least_mut:
    refine = True

while refine:
    refine = False
    tmp_col = np.where(class_label == tmp_grp[0][0])[0]
    for i in range(len(tmp_col)):
        if tmp_col[i] != 0 and tmp_col[i] != No_mutation - 1:
            tmp_diff = np.abs(np.append(np.append(diff[0:tmp_col[i], tmp_col[i]].T.ravel(), 100),
                                        diff[tmp_col[i], (tmp_col[i] + 1):No_mutation].ravel()))
            tmp_diff[tmp_col] += 100
            diff[0:tmp_col[i], tmp_col[i]] = tmp_diff[0:tmp_col[i]]
            diff[tmp_col[i], (tmp_col[i] + 1):No_mutation] = tmp_diff[(tmp_col[i] + 1):No_mutation]
        elif tmp_col[i] == 0:
            tmp_diff = np.append(100, diff[0, 1:No_mutation])
            tmp_diff[tmp_col] += 100
            diff[0, 1:No_mutation] = tmp_diff[1:No_mutation]
        else:
            tmp_diff = np.append(diff[0:(No_mutation - 1), No_mutation - 1], 100)
            tmp_diff[tmp_col] += 100
            diff[0:(No_mutation - 1), No_mutation - 1] = tmp_diff[0:(No_mutation - 1)]
        ind = tmp_diff.argmin()
        group_size[class_label.astype(np.int64, copy=False)[tmp_col[i]]] -= 1
        class_label[tmp_col[i]] = class_label[ind]
        group_size[class_label.astype(np.int64, copy=False)[tmp_col[i]]] += 1
    tmp_size = np.min(np.array(group_size)[np.array(group_size) > 0])
    tmp_grp = np.where(group_size == tmp_size)
    refine = False
    if tmp_size < least_mut:
        refine = True

labels = np.unique(class_label)
phi_out = np.zeros((len(labels), M))
for i in range(len(labels)):
    ind = np.where(class_label == labels[i])[0]
    class_label[ind] = i
    phi_out[i, :] = np.sum(phi_hat[ind, : ] * n[ind, : ], axis=0) / np.sum(n[ind, : ], axis=0)

if len(labels) > 1:
    sort_phi = sort_by_2norm(phi_out)
    phi_diff = sort_phi[1:, :] - sort_phi[:-1, :]
    min_ind, min_val = find_min_row_by_2norm(phi_diff)
    while np.linalg.norm(min_val) < least_diff:
        combine_ind = np.where(phi_out == sort_phi[min_ind, ])[0]
        combine_to_ind = np.where(phi_out == sort_phi[min_ind + 1, ])[0]
        class_label[class_label == combine_ind] = combine_to_ind
        labels = np.unique(class_label)
        phi_out = np.zeros((len(labels), M))
        for i in range(len(labels)):
            ind = np.where(class_label == labels[i])[0]
            class_label[ind] = i
            phi_out[i] = np.sum(phi_hat[ind, : ] * n[ind, : ], axis=0) / np.sum(n[ind, : ], axis=0)
        if len(labels) == 1:
            break
        else:
            sort_phi = sort_by_2norm(phi_out)
            phi_diff = sort_phi[1:, :] - sort_phi[:-1, :]
            min_ind, min_val = find_min_row_by_2norm(phi_diff)
phi_res = np.zeros((No_mutation, M))
for lab in range(np.shape(phi_out)[0]):
    phi_res[class_label == lab, ] = phi_out[lab, ]
class_label


ADMM finished.



array([0., 8., 1., ..., 9., 9., 9.])

In [ ]:
class_label

array([0., 8., 1., ..., 9., 9., 9.])